In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from citipy import citipy
import json

# Range of latitudes and longitudes
lat_range = (64, 20)
lng_range = (-154, -65)

# Import API key
from api_keys import weather_key

In [36]:
weather_data_df = pd.read_csv("data/noaa_storm_data.csv")
weather_data_df.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER
0,866322,CARTERET CO.,TRUTTNEYS LNDG,11/1/2019,15,Tornado,,EF0,0,0,...,1,NNW,TRUTTNEYS LNDG,34.7394,-77.1094,34.7411,-77.1053,A brief tornado occurred near the intersection...,A strong cold front moved across Eastern North...,1
1,859389,MARTIN CO.,WAVELAND,11/5/2019,1620,Tornado,,EF0,0,0,...,2,ENE,WAVELAND,27.2140,-80.1729,27.2123,-80.1766,Martin County Beach Patrol reported a waterspo...,A strong thunderstorm produced a waterspout ju...,2
2,864077,MOBILE CO.,PRICHARD,11/7/2019,1335,Tornado,,EF0,0,0,...,0,SSE,PRICHARD,30.7248,-88.0548,30.7260,-88.0486,The storm survey team surveyed the areas aroun...,Thunderstorms moved across the area during the...,3
3,857862,FRANKLIN PAR.,BASKIN,11/26/2019,1803,Tornado,,EF2,1,1,...,4,NE,BASKIN,32.2351,-91.7783,32.3184,-91.7095,This tornado began on Roger Miller Road as it ...,"Warm, moist air was in place across northeast ...",4
4,857944,CLAIBORNE PAR.,MAHAN,11/26/2019,1820,Tornado,,EF1,0,0,...,2,NNE,MAHAN,32.9017,-93.0423,32.9076,-93.0343,An EF-1 tornado with estimated maximum winds a...,A deep southwesterly flow aloft ahead of a lar...,5


In [37]:
tornado_df = weather_data_df.loc[:,["BEGIN_LOCATION","BEGIN_DATE", "EVENT_TYPE", "TOR_F_SCALE", "DEATHS_DIRECT", "END_LOCATION"]]
tornado_df["BEGIN_LAT"] = ""
tornado_df["BEGIN_LON"] = ""
tornado_df["END_LAT"] = ""
tornado_df["END_LON"] = ""
tornado_df["temp_max"] = ""
tornado_df["temp_max"] = ""
tornado_df["humidity"] = ""
tornado_df.head()

,BEGIN_LOCATION,BEGIN_DATE,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,temp_max,humidity
0,TRUTTNEYS LNDG,11/1/2019,Tornado,EF0,0,TRUTTNEYS LNDG,,,,,,
1,WAVELAND,11/5/2019,Tornado,EF0,0,WAVELAND,,,,,,
2,PRICHARD,11/7/2019,Tornado,EF0,0,PRICHARD,,,,,,
3,BASKIN,11/26/2019,Tornado,EF2,1,BASKIN,,,,,,
4,MAHAN,11/26/2019,Tornado,EF1,0,MAHAN,,,,,,


In [38]:
gmaps.configure(api_key = weather_key)

In [39]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

527

In [40]:
for index, row in tornado_df.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=04c41275fea3e8291170fd88095dde0f"
    beg_lat= row['BEGIN_LAT']
    beg_lon= row['BEGIN_LON']

In [41]:
cities_lat_lng = requests.get(base_url)

In [43]:
cities_lat_lng = cities_lat_lng.json()

tornado_df.loc[index, "Lat"] = cities_lat_lng["BEGIN_LAT"][0]["geometry"]["location"]["lat"]
tornado_df.loc[index, "Lng"] = cities_lat_lng["BEGIN_LON"][0]["geometry"]["location"]["lng"]
tornado_df.head()

AttributeError: 'dict' object has no attribute 'json'

In [28]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=04c41275fea3e8291170fd88095dde0f"
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for x, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (x % 50 == 0 and x >= 50):
        set_count += 1
        record_count = 0
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]
        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | sitka
Processing Record 2 of Set 1 | constitucion
Processing Record 3 of Set 1 | sterling
Processing Record 4 of Set 1 | anaconda
Processing Record 5 of Set 1 | san quintin
Processing Record 6 of Set 1 | norman wells
Processing Record 7 of Set 1 | monticello
Processing Record 8 of Set 1 | bainbridge
Processing Record 9 of Set 1 | la ronge
Processing Record 10 of Set 1 | michigan city
Processing Record 11 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 12 of Set 1 | anchorage
Processing Record 13 of Set 1 | fortuna
Processing Record 14 of Set 1 | palmer
Processing Record 15 of Set 1 | thompson
Processing Record 16 of Set 1 | panama city
Processing Record 17 of Set 1 | juneau
Processing Record 18 of Set 1 | camalu
Processing Record 19 of Set 1 | cockburn town
Processing Record 20 of Set 1 | lompoc
Processing Record 21 of Set 1 | ketchikan
Processing Record 22 of Set 1 | santa 

Processing Record 41 of Set 4 | marquette
Processing Record 42 of Set 4 | altamira
Processing Record 43 of Set 4 | soto la marina
Processing Record 44 of Set 4 | easton
Processing Record 45 of Set 4 | warrington
Processing Record 46 of Set 4 | bartlesville
Processing Record 47 of Set 4 | clarksdale
Processing Record 48 of Set 4 | yellowknife
Processing Record 49 of Set 4 | fort frances
Processing Record 0 of Set 5 | valdosta
Processing Record 1 of Set 5 | hurricane
Processing Record 2 of Set 5 | fairbanks
Processing Record 3 of Set 5 | jacksonville
Processing Record 4 of Set 5 | pearl river
Processing Record 5 of Set 5 | slave lake
Processing Record 6 of Set 5 | high level
Processing Record 7 of Set 5 | dawson creek
Processing Record 8 of Set 5 | magalia
Processing Record 9 of Set 5 | cerritos
Processing Record 10 of Set 5 | chapleau
Processing Record 11 of Set 5 | sonoita
Processing Record 12 of Set 5 | stephenville
Processing Record 13 of Set 5 | valle hermoso
Processing Record 14 of

Processing Record 35 of Set 8 | nogales
Processing Record 36 of Set 8 | morro bay
Processing Record 37 of Set 8 | terrace bay
Processing Record 38 of Set 8 | whitefish
Processing Record 39 of Set 8 | altus
Processing Record 40 of Set 8 | glendive
Processing Record 41 of Set 8 | ruston
Processing Record 42 of Set 8 | west fargo
Processing Record 43 of Set 8 | louiseville
Processing Record 44 of Set 8 | billings
Processing Record 45 of Set 8 | lac-megantic
Processing Record 46 of Set 8 | brewster
Processing Record 47 of Set 8 | nuevo progreso
Processing Record 48 of Set 8 | kantunilkin
Processing Record 49 of Set 8 | worland
Processing Record 0 of Set 9 | los palacios
Processing Record 1 of Set 9 | wawa
Processing Record 2 of Set 9 | socorro
Processing Record 3 of Set 9 | nuevitas
Processing Record 4 of Set 9 | stevens point
Processing Record 5 of Set 9 | beatrice
Processing Record 6 of Set 9 | elliot lake
Processing Record 7 of Set 9 | elwood
Processing Record 8 of Set 9 | glendale
Proc

In [16]:
city_data_pd = pd.DataFrame(city_data)
city_data_pd.count()


City          526
Lat           526
Lng           526
Max Temp      526
Humidity      526
Cloudiness    526
Wind Speed    526
Country       526
Date          526
dtype: int64

In [17]:
city_data_pd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,elko,41.0002,-115.5012,34.00,33,100,4.76,US,1615343671
1,clarksville,36.5298,-87.3595,62.60,75,1,5.75,US,1615343770
2,sooke,48.3746,-123.7276,46.40,79,56,13.80,CA,1615344205
3,choix,26.7092,-108.3219,75.83,20,81,2.26,MX,1615344206
4,hay river,60.8156,-115.7999,-5.80,77,100,5.75,CA,1615343664


In [18]:
america_city = city_data_pd[(city_data_pd["Country"]=="US")]
america_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,elko,41.0002,-115.5012,34.00,33,100,4.76,US,1615343671
1,clarksville,36.5298,-87.3595,62.60,75,1,5.75,US,1615343770
5,kenai,60.5544,-151.2583,33.80,86,90,9.22,US,1615344206
6,saint pete beach,27.7253,-82.7412,66.20,63,1,18.41,US,1615344206
9,hilo,19.7297,-155.0900,71.60,88,90,3.44,US,1615344199
...,...,...,...,...,...,...,...,...,...
520,eunice,30.4944,-92.4176,64.40,72,1,6.91,US,1615344317
522,parsons,37.3403,-95.2611,64.40,63,90,24.16,US,1615344318
523,polson,47.6936,-114.1632,30.99,68,97,1.99,US,1615344318
524,ruidoso,33.3317,-105.6730,53.60,32,90,23.02,US,1615343724
